# Data Cleaning Pipeline: Inside Airbnb Listings

This notebook executes a rigorous data cleaning workflow on the [**Inside Airbnb**](https://insideairbnb.com/get-the-data/) `listings.csv` dataset. The raw data contains significant noise, including unformatted currency strings, nested JSON-like amenities, and inconsistent categorical flags. Our objective is to transform this raw scrape into a structured, analysis-ready format optimized for machine learning and geospatial modeling.

## Import libraries and load dataset

In [1]:
import pandas as pd
import re

In [2]:
date_columns = [
    "last_scraped", "host_since", "calendar_last_scraped", 
    "first_review", "last_review"
]

zip_listings_df = pd.read_csv("listings.csv.gz", parse_dates=date_columns)
zip_listings_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13913,https://www.airbnb.com/rooms/13913,20250914034649,2025-09-16,city scrape,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730,...,4.87,4.78,4.78,NaN,f,2,1,1,0,0.30
1,15400,https://www.airbnb.com/rooms/15400,20250914034649,2025-09-16,city scrape,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,https://a0.muscache.com/pictures/428392/462d26...,60302,...,4.84,4.93,4.74,NaN,f,1,1,0,0,0.51
2,17402,https://www.airbnb.com/rooms/17402,20250914034649,2025-09-16,city scrape,Very Central Modern 3-Bed/2 Bath By Oxford St W1,"You'll have a great time in this beautiful, cl...","Fitzrovia is a very desirable trendy, arty and...",https://a0.muscache.com/pictures/39d5309d-fba7...,67564,...,4.72,4.89,4.61,NaN,f,2,2,0,0,0.32
3,24328,https://www.airbnb.com/rooms/24328,20250914034649,2025-09-18,previous scrape,Battersea live/work artist house,"Artist house by SW Battersea Park, bright high...","- Battersea is a quiet family area, easy acces...",https://a0.muscache.com/pictures/9194b40f-c627...,41759,...,4.93,4.60,4.65,NaN,f,1,1,0,0,0.53
4,36274,https://www.airbnb.com/rooms/36274,20250914034649,2025-09-15,city scrape,Bright 1 bedroom apt off brick lane in Shoreditch,*Update June '25- Pump Installed to improve wa...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,133271,...,4.46,4.85,4.54,NaN,t,2,2,0,0,0.09


In [3]:
zip_listings_df.shape

(96871, 79)

In [4]:
zip_listings_df.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'ca

## Classify columns into numeric, categorical, and other values

In [5]:
# 1. URL COLUMNS (Usually dropped or stored separately)
url_columns = [
    "listing_url", "picture_url", "host_url", 
    "host_thumbnail_url", "host_picture_url"
]

# 2. NUMERIC COLUMNS (Continuous and discrete numbers)
numeric_columns = [
    "id", "scrape_id", "host_id", "latitude", "longitude", 
    "accommodates", "bathrooms", "bedrooms", "beds", "price",
    "minimum_nights", "maximum_nights", "minimum_minimum_nights", 
    "maximum_minimum_nights", "minimum_maximum_nights", "maximum_maximum_nights", 
    "minimum_nights_avg_ntm", "maximum_nights_avg_ntm", "availability_30", 
    "availability_60", "availability_90", "availability_365", 
    "number_of_reviews", "number_of_reviews_ltm", "number_of_reviews_l30d", 
    "review_scores_rating", "review_scores_accuracy", "review_scores_cleanliness", 
    "review_scores_checkin", "review_scores_communication", 
    "review_scores_location", "review_scores_value", "reviews_per_month",
    "calculated_host_listings_count", "calculated_host_listings_count_entire_homes", 
    "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms",
    "availability_eoy", "estimated_occupancy_l365d",
    "estimated_revenue_l365d", "number_of_reviews_ly", "host_response_rate",
    "host_acceptance_rate", "host_listings_count", "host_total_listings_count",
    
]

# 3. CATEGORICAL COLUMNS (Nominal, Ordinal, and Boolean)
categorical_columns = [
    "source", "host_location", "host_response_time",
    "host_is_superhost", 
    "host_neighbourhood", 
    "host_has_profile_pic", "host_identity_verified", "neighbourhood", 
    "neighbourhood_cleansed", "neighbourhood_group_cleansed", "property_type", 
    "room_type", "bathrooms_text", "has_availability", "calendar_updated", 
    "license", "instant_bookable"
]

# 4. TEXT COLUMNS (Long-form unstructured text for NLP)
text_columns = [
    "name", "description", "neighborhood_overview", "host_about","host_name"
]

# 5. DATE COLUMNS (Require conversion to datetime objects)
date_columns = [
    "last_scraped", "host_since", "calendar_last_scraped", 
    "first_review", "last_review"
]

# 6. COMPLEX/JSON COLUMNS (Require parsing/flattening)
complex_columns = [
    "host_verifications", "amenities"
]


## Check for duplicated listing ids

In [6]:
print(zip_listings_df.duplicated(subset=['id']).sum())

0


## Drop *_url columns

In [7]:
listings_df = zip_listings_df.drop(columns=url_columns)
listings_df.shape

(96871, 74)

## Numeric columns

In [8]:
listings_df[numeric_columns].dtypes

id                                                int64
scrape_id                                         int64
host_id                                           int64
latitude                                        float64
longitude                                       float64
accommodates                                      int64
bathrooms                                       float64
bedrooms                                        float64
beds                                            float64
price                                            object
minimum_nights                                    int64
maximum_nights                                    int64
minimum_minimum_nights                          float64
maximum_minimum_nights                          float64
minimum_maximum_nights                          float64
maximum_maximum_nights                          float64
minimum_nights_avg_ntm                          float64
maximum_nights_avg_ntm                          

## Transform `price` and other numeric columns

In [9]:
listings_df['price'].unique()

array(['$70.00', '$149.00', '$411.00', ..., '$10,028.00', '$10,020.00',
       '$3,509.00'], shape=(1351,), dtype=object)

In [10]:
listings_df['price'] = listings_df['price'].str.replace(r'[$,]', '', regex=True).astype(float)

In [11]:
listings_df['price'].unique()

array([   70.,   149.,   411., ..., 10028., 10020.,  3509.], shape=(1351,))

In [12]:
listings_df['host_response_rate'] = listings_df['host_response_rate'].str.replace(r'[%]', '', regex=True).astype(float)
listings_df['host_acceptance_rate'] = listings_df['host_acceptance_rate'].str.replace(r'[%]', '', regex=True).astype(float)

listings_df['host_total_listings_count'] = pd.to_numeric(listings_df['host_total_listings_count'], errors='coerce')
listings_df['host_listings_count'] = pd.to_numeric(listings_df['host_listings_count'], errors='coerce')

## Categorical columns

In [13]:
listings_df[categorical_columns].dtypes

source                           object
host_location                    object
host_response_time               object
host_is_superhost                object
host_neighbourhood               object
host_has_profile_pic             object
host_identity_verified           object
neighbourhood                    object
neighbourhood_cleansed           object
neighbourhood_group_cleansed    float64
property_type                    object
room_type                        object
bathrooms_text                   object
has_availability                 object
calendar_updated                float64
license                         float64
instant_bookable                 object
dtype: object

## Transform boolean values

In [14]:
listings_df['host_is_superhost'] = listings_df['host_is_superhost'].map({'t': True, 'f': False})
listings_df["host_has_profile_pic"] = listings_df['host_has_profile_pic'].map({'t': True, 'f': False})
listings_df["host_identity_verified"] = listings_df['host_identity_verified'].map({'t': True, 'f': False})
listings_df["instant_bookable"] = listings_df['instant_bookable'].map({'t': True, 'f': False})
listings_df["has_availability"] = listings_df['has_availability'].map({'t': True, 'f': False})

# bool_cols = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'instant_bookable', 'has_availability']

# listings_df[bool_cols] = listings_df[bool_cols].map({'t': True, 'f': False})

## Drop columns that contain all NA values

In [15]:
listings_df = listings_df.drop(columns="neighbourhood")
listings_df = listings_df.drop(columns="calendar_updated")
listings_df = listings_df.drop(columns="license")
listings_df = listings_df.drop(columns="neighbourhood_group_cleansed")

In [16]:
categorical_columns.remove("neighbourhood")
categorical_columns.remove("calendar_updated")
categorical_columns.remove("license")
categorical_columns.remove("neighbourhood_group_cleansed")

## Text columns

In [17]:
print(listings_df[text_columns].dtypes)

listings_df[text_columns]

name                     object
description              object
neighborhood_overview    object
host_about               object
host_name                object
dtype: object


,name,description,neighborhood_overview,host_about,host_name
0,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,Finsbury Park is a friendly melting pot commun...,I am a Multi-Media Visual Artist and Creative ...,Alina
1,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,It is Chelsea.,"English, grandmother, I have travelled quite ...",Philippa
2,Very Central Modern 3-Bed/2 Bath By Oxford St W1,"You'll have a great time in this beautiful, cl...","Fitzrovia is a very desirable trendy, arty and...",We are Liz and Jack. We manage a number of ho...,Liz
3,Battersea live/work artist house,"Artist house by SW Battersea Park, bright high...","- Battersea is a quiet family area, easy acces...","I've been using Airbnb for a while now, both a...",Joe
4,Bright 1 bedroom apt off brick lane in Shoreditch,*Update June '25- Pump Installed to improve wa...,NaN,"We are Hendryks Services - your resident, mana...",Hendryks
...,...,...,...,...,...
96866,"Blueground | Finsbury, balcony, nr St. Paul's",Feel at home wherever you choose to live with ...,NaN,"We’re Blueground, a global proptech company wi...",Blueground
96867,Self-Contained Studio in Heart of Tooting Broa...,Welcome to your perfect London stay - The Lond...,NaN,NaN,Ali
96868,One bedroom apartment Dagenham,"Modern 1-bedroom apartment in Kerwin House, Da...",NaN,NaN,Arnelle
96869,Short Stay,Take it easy at this unique and tranquil getaway.,NaN,NaN,Tayba


## Date columns

In [18]:
print(listings_df[date_columns].dtypes)

listings_df[date_columns]

last_scraped             datetime64[ns]
host_since               datetime64[ns]
calendar_last_scraped    datetime64[ns]
first_review             datetime64[ns]
last_review              datetime64[ns]
dtype: object


,last_scraped,host_since,calendar_last_scraped,first_review,last_review
0,2025-09-16,2009-11-16,2025-09-16,2010-08-18,2025-08-21
1,2025-09-16,2009-12-05,2025-09-16,2009-12-21,2025-04-05
2,2025-09-16,2010-01-04,2025-09-16,2011-03-21,2024-02-19
3,2025-09-18,2009-09-28,2025-09-18,2010-11-15,2025-07-05
4,2025-09-15,2010-05-27,2025-09-15,2011-03-20,2025-09-06
...,...,...,...,...,...
96866,2025-09-16,2019-12-03,2025-09-16,NaT,NaT
96867,2025-09-15,2025-09-12,2025-09-15,2025-09-15,2025-09-15
96868,2025-09-14,2021-02-18,2025-09-14,NaT,NaT
96869,2025-09-15,2025-03-09,2025-09-15,NaT,NaT


## Complex columns

In [19]:
listings_df[complex_columns]

,host_verifications,amenities
0,"['email', 'phone']","[""Self check-in"", ""Kitchen"", ""Smoke alarm"", ""D..."
1,"['email', 'phone']","[""Kitchen"", ""Smoke alarm"", ""Dedicated workspac..."
2,"['email', 'phone', 'work_email']","[""Kitchen"", ""Cleaning products"", ""Smoke alarm""..."
3,"['email', 'phone', 'work_email']","[""Self check-in"", ""Kitchen"", ""Apple speaker Bl..."
4,"['email', 'phone']","[""Washer"", ""Kitchen"", ""TV with standard cable""..."
...,...,...
96866,"['email', 'phone', 'work_email']","[""Self check-in"", ""Kitchen"", ""Smoke alarm"", ""S..."
96867,"['email', 'phone']","[""First aid kit"", ""Washer"", ""TV"", ""Kitchen"", ""..."
96868,"['email', 'phone']","[""First aid kit"", ""TV"", ""Kitchen"", ""Free parki..."
96869,"['email', 'phone']","[""First aid kit"", ""Washer"", ""TV"", ""Kitchen"", ""..."


## Missing data in columns

In [20]:
pd.set_option('display.max_rows', None)

null_percentage = (listings_df.isnull().mean() * 100).sort_values(ascending=False)

print(null_percentage)

neighborhood_overview                           57.460953
host_neighbourhood                              52.669013
host_about                                      48.557360
beds                                            36.047940
estimated_revenue_l365d                         36.035552
price                                           36.035552
bathrooms                                       35.971550
host_response_rate                              32.731158
host_response_time                              32.731158
host_acceptance_rate                            28.656667
review_scores_location                          24.946578
review_scores_value                             24.946578
review_scores_checkin                           24.945546
review_scores_communication                     24.921803
review_scores_accuracy                          24.916642
review_scores_cleanliness                       24.910448
review_scores_rating                            24.901157
last_review   

## Save clean data after process

In [21]:
listings_df.to_csv("clean_listings.csv.gz", index =False)